In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/My\ Drive/Colab\ Notebooks/Transformer

/content/drive/My Drive/Colab Notebooks/Transformer


# ライブラリ読み込み

In [3]:
%%bash 
pip install SudachiPy
pip install SudachiDict-full
pip install sortedcontainers==2.1.0

  Created wheel for SudachiPy: filename=SudachiPy-0.5.1-cp36-cp36m-linux_x86_64.whl size=871065 sha256=5ed8f8a84cf45769664a62dddb084c08691f484d51c79a5d0caae0b6b6aafeac
  Stored in directory: /root/.cache/pip/wheels/a5/fb/54/82b45dd3d48476dfe6d055da5dee9d5fa1c1941368f9568e90
Successfully built SudachiPy
  Found existing installation: sortedcontainers 2.3.0
    Uninstalling sortedcontainers-2.3.0:
      Successfully uninstalled sortedcontainers-2.3.0
  Created wheel for SudachiDict-full: filename=SudachiDict_full-20201223.post1-cp36-none-any.whl size=126956073 sha256=b84ce831c52b18af1975e7c11b7fc19fd004afb7a92e0cc6da546513ff98786e
  Stored in directory: /root/.cache/pip/wheels/e4/b4/b1/8e5c8f1bb9052bb69259c9674b5cfb09a3bdceb01cb009e91a
Successfully built SudachiDict-full


In [4]:
!sudachipy link -t full

Set the default dictionary to `sudachidict_full`.


In [5]:
!pwd

/content/drive/My Drive/Colab Notebooks/Transformer


ランタイムを再駆動する

In [1]:
!pip install sortedcontainers==2.1.0

In [2]:
from sudachipy import tokenizer 
from sudachipy import dictionary
tokenizer_obj = dictionary.Dictionary().create()

In [24]:
import pandas as pd
import re
import codecs
import copy

# データ読み込み

In [22]:

corpus_path = './DATA/Kesennuma.csv'
df = pd.read_csv(corpus_path)
mode = tokenizer.Tokenizer.SplitMode.C
df['A'] = ""
print('**** Amount of data ****')
print(df)
print('\n')
print('**** Amount of data ****')
#for index, row in df.iterrows():
    #print(row['項目名'])


**** Amount of data ****
     市区町村                項目名       回答語形 A 
0    気仙沼市               起きない       オギネー   
1    気仙沼市                来ない        コネー   
2    気仙沼市                しない        シネー   
3    気仙沼市                しない        スネー   
4    気仙沼市             行かなかった     イガナカッタ   
..    ...                ...        ... ..
242  気仙沼市            知事：一つ仮名  ［ts??z??］   
243  気仙沼市            地図：一つ仮名   ［ts?z??］   
244  気仙沼市   切符（きっぷ）：拍意識・文字意識      き／っ／ぷ   
245  気仙沼市   切符（きっぷ）：拍意識・文字意識       きっ／ぷ   
246  気仙沼市  風船（ふうせん）：拍意識・文字意識    ふ／う／せ／ん   

[247 rows x 4 columns]


**** Amount of data ****


# 前処理

In [ ]:
c_path = './DATA/Kessen.txt'

with open(c_path, 'a') as f:
    for index, row in df.iterrows():
        s = re.sub('[〈]+.*[〉]+', '', row['項目名'], 2)
        s = re.sub('[（]+.{1,7}[）]+', '', s, 2)
        s = re.sub('[（]+.{1,8}[・]+.{1,8}[）]+', '', s, 2)
        s = re.sub('[：].*[音意語].*', '', s, 2)
        s = re.sub('[\u3041-\u309F]??[：]', '', s, 2)
        #print(s)
        tokens =  tokenizer_obj.tokenize( s, mode)
        parsed = ""
        for t in tokens:
            parsed +=  str(t.reading_form())
        if parsed == "" :
            parsed = row['項目名']
        print(parsed,file=f)

# モデル定義

In [32]:
c_path = './DATA/Kessen.txt'

c_df = copy.deepcopy(df)

with open(c_path, 'a') as f:
    for index, row in c_df.iterrows():
        s = re.sub('[〈]+.*[〉]+', '', row['項目名'], 2)
        s = re.sub('[（]+.{1,7}[）]+', '', s, 2)
        s = re.sub('[（]+.{1,8}[・]+.{1,8}[）]+', '', s, 2)
        s = re.sub('[：].*[音意語].*', '', s, 2)
        s = re.sub('[\u3041-\u309F]??[：]', '', s, 2)
        #print(s)
        tokens =  tokenizer_obj.tokenize( s, mode)
        parsed = ""
        for t in tokens:
            parsed +=  str(t.reading_form())
        if parsed == "" :
            parsed = row['項目名']
        row['A'] = parsed
        
        #print(parsed,file=f)
print(c_df.head())
c_df.to_csv('./DATA/kesen2.csv')

   市区町村     項目名    回答語形       A
0  気仙沼市    起きない    オギネー    オキナイ
1  気仙沼市     来ない     コネー     コナイ
2  気仙沼市     しない     シネー     シナイ
3  気仙沼市     しない     スネー     シナイ
4  気仙沼市  行かなかった  イガナカッタ  イカナカッタ


# 学習実行